# This colab notebook must be run on a **P100** GPU or **T4** GPU instance 

Cell-1: Ensure the required gpu instance (P100 or T4)

In [ ]:
#no.of sockets i.e available slots for physical processors
!lscpu | grep 'Socket(s):'
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'
#GPU count and name
!nvidia-smi -L
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Socket(s):           1
Core(s) per socket:  1
Thread(s) per core:  2
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-076b5f5f-c889-3069-967f-b2c54c5de3cd)
Wed Oct  7 10:21:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+---------------------

Cell-2: Add Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Cell-3: Install Required Dependencies

In [ ]:
!pip install efficientnet_pytorch==0.7.0 
!pip install albumentations==0.4.5
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch\_stable.html -q\

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=1b91a0113310e84891d4c7b9e039a5328474099da4030d4fb1179f8e1e044407
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 122kB 4.6MB/s 
     |████████████████████████████████| 634kB 13.6MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=2b5c44c98073bc2b77c14395df09a518b8d128bbea69fcc8fc797e6792fc56b4
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=e4845c32e947bbf0cd73402f95e7e109367ee0f9ed2118bf6652874e4deeb347
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentati

Cell-4: Import necessary libraries

In [ ]:
import sys
sys.path.insert(0, "/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/src_ensemble")

from dataset import *
from model import *
from utils import *
from torch.utils.data import DataLoader

set_random_state(4690)
imgs = np.load('/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/zindi_npy_data/train_imgs.npy')
labels_quality = np.load('/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/zindi_npy_data/train_labels_quality.npy')
imgs = imgs[labels_quality == 1]
test_dataset = ZCTESTDataset(imgs)                                           
test_loader = DataLoader( 
                  test_dataset, 
                  batch_size=86, 
                  shuffle=False, 
                  num_workers=2,
                  drop_last=False,
                  pin_memory=True,                  
                  )

Cell-5: Run this cell to generate lq2 only b2 step1 pred

In [ ]:
from tqdm import tqdm

model_infos = {
      # b2 lq2 only step 1 
      'b2_lq2_only_step1_fold0': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b2_step1/weights/fold0/checkpoint_best_f1_score_fold0.pth',
            'model_name': 'efficientnet-b2',
            'global_dim': 1408,
            'version': 1,
      },
      'b2_lq2_only_step1_fold1': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b2_step1/weights/fold1/checkpoint_best_f1_score_fold1.pth',
            'model_name': 'efficientnet-b2',
            'global_dim': 1408,
            'version': 1,
      },
      'b2_lq2_only_step1_fold2': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b2_step1/weights/fold2/checkpoint_best_f1_score_fold2.pth',
            'model_name': 'efficientnet-b2',
            'global_dim': 1408,
            'version': 1,
      },
      'b2_lq2_only_step1_fold3': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b2_step1/weights/fold3/checkpoint_best_f1_score_fold3.pth',
            'model_name': 'efficientnet-b2',
            'global_dim': 1408,
            'version': 1,
      },
      'b2_lq2_only_step1_fold4': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b2_step1/weights/fold4/checkpoint_best_f1_score_fold4.pth',
            'model_name': 'efficientnet-b2',
            'global_dim': 1408,
            'version': 1,
      },
}

models = []
for model_type in model_infos:
    try:
        models.append(get_model(model_infos[model_type]))
    except:
        print('Error has occured...skipping!')

all_growth_stage = []
torch.set_grad_enabled(False)                                
with torch.no_grad():
    for itera_no, data in tqdm(enumerate(test_loader), total=len(test_loader)):
        images, images_fliplr = data
        images = images.cuda()
        images_fliplr = images_fliplr.cuda()

        y_pred = 0                                                      
        for i in range(len(models)):
            with torch.cuda.amp.autocast():
               out_1 = models[i](images)
               out_2 = models[i](images_fliplr)
            y_pred += out_1['LOGITS'][:,0].cpu().data.numpy()
            y_pred += out_2['LOGITS'][:,0].cpu().data.numpy()

        y_pred = y_pred/(len(models)*2)
        all_growth_stage += y_pred.clip(min=0, max=6).tolist()      
            
all_growth_stage = np.array(all_growth_stage)
np.save('/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/b2_lq2_only_step1_5fold_pred_wd.npy', all_growth_stage)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b1-f1951068.pth



Loaded pretrained weights for efficientnet-b1
(Val) loss is 3.9425562501162448
(Val) f1 score is 0.0
(Val) rmse is 3.19597281310445
(Train) loss is 3.140910883744558
(Train) f1 score is 0
(Train) rmse is 0
Loaded pretrained weights for efficientnet-b1
(Val) loss is 3.178070466697947
(Val) f1 score is 0.0022675736961451248
(Val) rmse is 3.03745487033048
(Train) loss is 3.0040662388006845
(Train) f1 score is 0
(Train) rmse is 0
Loaded pretrained weights for efficientnet-b1
(Val) loss is 3.5846992015003534
(Val) f1 score is 0.0
(Val) rmse is 3.186018693259899
(Train) loss is 3.1435382465521493
(Train) f1 score is 0
(Train) rmse is 0
Loaded pretrained weights for efficientnet-b1
(Val) loss is 3.062673478953266
(Val) f1 score is 0.005952380952380952
(Val) rmse is 3.0329025835688332
(Train) loss is 3.1356143852074942
(Train) f1 score is 0
(Train) rmse is 0
Loaded pretrained weights for efficientnet-b1
(Val) loss is 2.9511563928240028
(Val) f1 score is 0.004608294930875576
(Val) rmse is 2.90

100%|██████████| 92/92 [04:10<00:00,  2.72s/it]


Cell-5: Run this cell to generate lq2 only b4 step1 pred

In [ ]:
from tqdm import tqdm

model_infos = {
      # b4 lq2 only step 1 
      'b4_lq2_only_step1_fold0': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b4_step1/weights/fold0/checkpoint_best_f1_score_fold0.pth',
            'model_name': 'efficientnet-b4',
            'global_dim': 1792,
            'version': 1,
      },
      'b4_lq2_only_step1_fold1': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b4_step1/weights/fold1/checkpoint_best_f1_score_fold1.pth',
            'model_name': 'efficientnet-b4',
            'global_dim': 1792,
            'version': 1,
      },
      'b4_lq2_only_step1_fold2': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b4_step1/weights/fold2/checkpoint_best_f1_score_fold2.pth',
            'model_name': 'efficientnet-b4',
            'global_dim': 1792,
            'version': 1,
      },
      'b4_lq2_only_step1_fold3': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b4_step1/weights/fold3/checkpoint_best_f1_score_fold3.pth',
            'model_name': 'efficientnet-b4',
            'global_dim': 1792,
            'version': 1,
      },
      'b4_lq2_only_step1_fold4': {
            'path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_lq2_only_effnet_b4_step1/weights/fold4/checkpoint_best_f1_score_fold4.pth',
            'model_name': 'efficientnet-b4',
            'global_dim': 1792,
            'version': 1,
      },
}

models = []
for model_type in model_infos:
    try:
        models.append(get_model(model_infos[model_type]))
    except:
        print('Error has occured...skipping!')

all_growth_stage = []
torch.set_grad_enabled(False)                                
with torch.no_grad():
    for itera_no, data in tqdm(enumerate(test_loader), total=len(test_loader)):
        images, images_fliplr = data
        images = images.cuda()
        images_fliplr = images_fliplr.cuda()

        y_pred = 0                                                      
        for i in range(len(models)):
            with torch.cuda.amp.autocast():
               out_1 = models[i](images)
               out_2 = models[i](images_fliplr)
            y_pred += out_1['LOGITS'][:,0].cpu().data.numpy()
            y_pred += out_2['LOGITS'][:,0].cpu().data.numpy()

        y_pred = y_pred/(len(models)*2)
        all_growth_stage += y_pred.clip(min=0, max=6).tolist()      
            
all_growth_stage = np.array(all_growth_stage)
np.save('/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/b4_lq2_only_step1_5fold_pred_wd.npy', all_growth_stage)